In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

C:\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import zipfile
import pandas as pd
 
# read the dataset using the compression zip
df = pd.read_csv('Audiobooks Data.zip',compression='zip')

In [4]:
df.head()

,00994,1620,1620.1,19.73,19.73.1,1,10.00,0.99,1603.80,5,92,0
0,1143,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,0,0
1,2059,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,388,0
2,2882,1620.0,1620,5.96,5.96,0,8.91,0.42,680.4,1,129,0
3,3342,2160.0,2160,5.33,5.33,0,8.91,0.22,475.2,0,361,0
4,3416,2160.0,2160,4.61,4.61,0,8.91,0.00,0.0,0,0,0


In [48]:
df.drop([ '1'], axis=1, inplace=True)


In [49]:
df.head()

,00994,1620,1620.1,19.73,10.00,0.99,1603.80,5,92,0
0,1143,2160.0,2160,5.33,8.91,0.00,0.0,0,0,0
1,2059,2160.0,2160,5.33,8.91,0.00,0.0,0,388,0
2,2882,1620.0,1620,5.96,8.91,0.42,680.4,1,129,0
3,3342,2160.0,2160,5.33,8.91,0.22,475.2,0,361,0
4,3416,2160.0,2160,4.61,8.91,0.00,0.0,0,0,0


In [50]:
df.isnull().sum()

00994      0
1620       0
1620.1     0
19.73      0
10.00      0
0.99       0
1603.80    0
5          0
92         0
0          0
dtype: int64

In [54]:
# Extracting target columns
target_columns = ["0"]
target = df[target_columns]

# Extracting feature columns
feature_columns = [col for col in df.columns if col not in target_columns]
features = df[feature_columns]


In [55]:
print(features)

       00994    1620  1620.1  19.73  10.00  0.99  1603.80  5   92
0       1143  2160.0    2160   5.33   8.91  0.00      0.0  0    0
1       2059  2160.0    2160   5.33   8.91  0.00      0.0  0  388
2       2882  1620.0    1620   5.96   8.91  0.42    680.4  1  129
3       3342  2160.0    2160   5.33   8.91  0.22    475.2  0  361
4       3416  2160.0    2160   4.61   8.91  0.00      0.0  0    0
...      ...     ...     ...    ...    ...   ...      ... ..  ...
14078  28220  1620.0    1620   5.33   9.00  0.61    988.2  0    4
14079  28671  1080.0    1080   6.55   6.00  0.29    313.2  0   29
14080  31134  2160.0    2160   6.14   8.91  0.00      0.0  0    0
14081  32832  1620.0    1620   5.33   8.00  0.38    615.6  0   90
14082    251  1674.0    3348   5.33   8.91  0.00      0.0  0    0

[14083 rows x 9 columns]


In [56]:
import numpy as np

# Assuming target is your Pandas DataFrame or Series
num_one_targets = int(np.sum(target.values))
zero_target_counter = 0
indices_to_remove = []

if num_one_targets > 0:
    for i in range(target.shape[0]):
        if (target.iloc[i] == 0).any():  # Use .any() to check if any element is equal to 0
            zero_target_counter += 1
        if zero_target_counter > num_one_targets:
            indices_to_remove.append(i)


In [57]:
target_balanced = target.drop(indices_to_remove)
print(target_balanced)

      0
0     0
1     0
2     0
3     0
4     0
...  ..
2868  0
2869  0
2870  1
2871  0
2872  0

[2873 rows x 1 columns]


In [58]:
from sklearn import preprocessing
from sklearn.utils import shuffle

# Assuming features is your feature DataFrame
features_std = preprocessing.scale(features)

# Assuming target is your target DataFrame
# Replace 'your_target_column' with the actual name of your target column
shuffled_data = pd.concat([pd.DataFrame(features_std, columns=features.columns), target], axis=1)

# Shuffle the data
shuffled_data = shuffle(shuffled_data, random_state=42)


In [59]:
shuffled_data.head()

,00994,1620,1620.1,19.73,10.00,0.99,1603.80,5,92,0
327,1.510943,1.127651,0.735124,3.751549,0.000439,-0.272081,-0.162280,-0.14856,-0.509401,0
903,-1.034333,-0.799618,-0.749212,-0.359575,0.000439,-0.520945,-0.511708,-0.14856,-0.475390,0
4043,-0.552437,-0.799618,-0.749212,0.147470,0.000439,-0.520945,-0.511708,-0.14856,-0.702129,0
3571,-0.572147,-0.799618,-0.749212,-0.124306,0.000439,-0.520945,-0.511708,-0.14856,-0.702129,0
8492,0.366570,-0.799618,-0.749212,-0.302786,0.000439,-0.520945,-0.511708,-0.14856,-0.350684,0


In [62]:
from sklearn.model_selection import train_test_split

# Assuming shuffled_data is your shuffled DataFrame with standardized features and target
# Replace 'your_target_column' with the actual name of your target column
X = shuffled_data.drop(["0"], axis=1)  # Features
y = shuffled_data[["0"]]  # Target

# Split the data into train, validation, and test sets
# Adjust the test_size and validation_size as needed
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [66]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
from sklearn.metrics import accuracy_score

# Assuming you have X_train, X_val, y_train, and y_val from the previous split
# Replace 'your_target_column' with the actual name of your target column

# Define the model
model = Sequential()
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1, activation='sigmoid'))


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), verbose=2)

# Evaluate on the test set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Test Accuracy: {accuracy}')


Epoch 1/100
353/353 - 3s - loss: 0.6333 - accuracy: 0.8790 - val_loss: 0.3368 - val_accuracy: 0.9062 - 3s/epoch - 9ms/step
Epoch 2/100
353/353 - 1s - loss: 0.3311 - accuracy: 0.8948 - val_loss: 0.2963 - val_accuracy: 0.9055 - 1s/epoch - 3ms/step
Epoch 3/100
353/353 - 1s - loss: 0.3133 - accuracy: 0.8955 - val_loss: 0.2888 - val_accuracy: 0.9070 - 1s/epoch - 3ms/step
Epoch 4/100
353/353 - 1s - loss: 0.3077 - accuracy: 0.8952 - val_loss: 0.2839 - val_accuracy: 0.9091 - 1s/epoch - 3ms/step
Epoch 5/100
353/353 - 1s - loss: 0.3007 - accuracy: 0.8985 - val_loss: 0.2859 - val_accuracy: 0.9112 - 1s/epoch - 3ms/step
Epoch 6/100
353/353 - 1s - loss: 0.2988 - accuracy: 0.8987 - val_loss: 0.2790 - val_accuracy: 0.9098 - 1s/epoch - 4ms/step
Epoch 7/100
353/353 - 2s - loss: 0.2965 - accuracy: 0.8993 - val_loss: 0.2863 - val_accuracy: 0.9077 - 2s/epoch - 6ms/step
Epoch 8/100
353/353 - 2s - loss: 0.2956 - accuracy: 0.8976 - val_loss: 0.2735 - val_accuracy: 0.9091 - 2s/epoch - 5ms/step
Epoch 9/100
353/

Epoch 68/100
353/353 - 1s - loss: 0.2679 - accuracy: 0.9023 - val_loss: 0.2486 - val_accuracy: 0.9119 - 1s/epoch - 3ms/step
Epoch 69/100
353/353 - 1s - loss: 0.2674 - accuracy: 0.9024 - val_loss: 0.2709 - val_accuracy: 0.9077 - 1s/epoch - 3ms/step
Epoch 70/100
353/353 - 1s - loss: 0.2691 - accuracy: 0.9010 - val_loss: 0.2429 - val_accuracy: 0.9134 - 1s/epoch - 3ms/step
Epoch 71/100
353/353 - 1s - loss: 0.2682 - accuracy: 0.9014 - val_loss: 0.2490 - val_accuracy: 0.9112 - 1s/epoch - 3ms/step
Epoch 72/100
353/353 - 1s - loss: 0.2678 - accuracy: 0.9024 - val_loss: 0.2429 - val_accuracy: 0.9112 - 1s/epoch - 3ms/step
Epoch 73/100
353/353 - 1s - loss: 0.2666 - accuracy: 0.9033 - val_loss: 0.2493 - val_accuracy: 0.9091 - 1s/epoch - 3ms/step
Epoch 74/100
353/353 - 1s - loss: 0.2675 - accuracy: 0.9013 - val_loss: 0.2469 - val_accuracy: 0.9084 - 1s/epoch - 3ms/step
Epoch 75/100
353/353 - 1s - loss: 0.2664 - accuracy: 0.9027 - val_loss: 0.2439 - val_accuracy: 0.9119 - 1s/epoch - 3ms/step
Epoch 76